In [67]:
import numpy as np
import pandas as pd
import tensorflow as tf


In [68]:
data = np.array(pd.read_csv("Dataset/Text_Emotion_Data.csv"))
stopwords = np.array(pd.read_csv("Dataset/stopwords.txt", sep=" ", header=None)).flatten()

In [69]:
X = data[:, 0]
y = data[:, 1]

In [70]:
# %pip install --upgrade --force-reinstall tensorflow

In [71]:
import re

def remove_none_alpha(x):
    regex = re.compile('[^a-zA-Z]')
    return regex.sub(' ', x)

In [72]:
# stopwords = [remove_none_alpha(w) for w in stopwords]

In [73]:
X_seq = []
for sentence in X:
    words_seq = remove_none_alpha(sentence).lower().split(" ")
    words_seq = [w for w in words_seq if len(w) > 2]
    words_seq = [w for w in words_seq if w not in stopwords]
    X_seq.append(words_seq)

In [74]:
max_len = len(max(X_seq, key=len))

# create a new list of lists with the desired size, padding with empty strings
sequences = [['' for _ in range(max_len)] for _ in range(len(X_seq))]

# copy the elements from the original lists into the new list, padding with empty strings as needed
for i, sequence in enumerate(X_seq):
    sequences[i][:len(sequence)] = sequence

In [75]:
import numpy as np

def one_hot_encode(sequence, word_to_idx):
    num_words = len(word_to_idx)
    encoding = np.zeros((len(sequence), num_words))
    for i, word in enumerate(sequence):
        if word in word_to_idx:
            encoding[i, word_to_idx[word]] = 1
    return encoding

def one_hot_encode2(sequence, word_to_idx, max_bit_num):
    encoding = np.zeros((len(sequence), max_bit_num))
    for i, word in enumerate(sequence):
        if word in word_to_idx:            
            binary_string = bin(word_to_idx[word])[2:]  # exclude the "0b" prefix
            binary_array = np.array([int(digit) for digit in binary_string])
            binary_array = np.pad(binary_array, (max_bit_num - len(binary_array), 0), 'constant')

            encoding[i, :] = binary_array
    return encoding

unique_words = list(np.unique([word for sequence in sequences for word in sequence]))
word_to_idx = {word: idx for idx, word in enumerate(unique_words)}

max_bit_num = 1
while(True):
    if(2**max_bit_num > len(unique_words)): break
    max_bit_num += 1

encoded_sequences = []
for sequence in sequences:
    encoded_sequences.append(one_hot_encode(sequence, word_to_idx))

encoded_sequences = np.array(encoded_sequences)

In [76]:
def select_test_dataset(x, y, _y):
    unique_classes = np.unique(_y)
    train_indices = []
    test_indices = []
    
    for cls in unique_classes:
        cls_indices = np.where(_y == cls)[0]
        train_indices.extend(cls_indices[:-150])
        test_indices.extend(cls_indices[-150:])
    
    train_x = x[train_indices]
    train_y = y[train_indices]
    test_x = x[test_indices]
    test_y = y[test_indices]
    
    return train_x, train_y, test_x, test_y

In [77]:
# X_train = encoded_sequences[:-150]
# X_test = encoded_sequences[-150:]
# # y_train_str = y[:-150]
# # y_test_str = y[-150:]

In [78]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
y_encoded = le.fit_transform(y)

In [79]:
# # Define the function you want to apply
# def to_binary(num):
#     binary_string = bin(num)[2:]  # exclude the "0b" prefix
#     binary_array = np.array([int(digit) for digit in binary_string])
#     binary_array = np.pad(binary_array, (3 - len(binary_array), 0), 'constant')
#     return binary_array

# # Create a NumPy array
# arr = np.array(y_encoded)

# # Initialize the result array
# y_encoded_onehot = np.empty((arr.shape[0], 3), dtype=int)

# # Apply the function on all elements of the array
# for i, num in enumerate(arr):
#     y_encoded_onehot[i] = to_binary(num)


In [80]:
min(y_encoded)

0

In [81]:
def to_binary(num):
    binary_array = np.zeros(5)
    binary_array[num] = 1
    return binary_array

# Create a NumPy array
arr = np.array(y_encoded)

# Initialize the result array
y_encoded_onehot = np.empty((arr.shape[0], 5), dtype=int)

# Apply the function on all elements of the array
for i, num in enumerate(arr):
    y_encoded_onehot[i] = to_binary(num)

In [82]:
y_encoded_onehot = tf.keras.utils.to_categorical(y_encoded, num_classes = 5)


In [83]:
X_train, y_train, X_test, y_test = select_test_dataset(encoded_sequences, y_encoded_onehot, y_encoded)

In [84]:
# y_train = y_encoded_onehot[:-150]
# y_test = y_encoded_onehot[-150:]

In [85]:
type(X_train)

numpy.ndarray

In [86]:
X_train.shape

(2500, 19, 6523)

In [87]:
# import tensorflow as tf

# # Define the Elman network architecture
# input_dim = X_train.shape[2]
# hidden_dim = 300
# output_dim = y_train.shape[1]
# num_steps = X_train.shape[1]

# inputs = tf.keras.layers.Input(shape=(num_steps, input_dim))
# hidden_layer = tf.keras.layers.SimpleRNN(hidden_dim, activation='tanh', return_sequences=True)(inputs)

# # Connect the hidden layer to itself at the previous time step
# hidden_layer_with_context = tf.keras.layers.SimpleRNN(hidden_dim, activation='tanh', return_sequences=True)(hidden_layer)
# hidden_layer_with_context = tf.keras.layers.Concatenate()([hidden_layer, hidden_layer_with_context])

# # Flatten the output of the hidden layer
# flatten_layer = tf.keras.layers.Flatten()(hidden_layer_with_context)

# outputs = tf.keras.layers.Dense(output_dim, activation='softmax')(flatten_layer)

# # Define the model
# model = tf.keras.Model(inputs=inputs, outputs=outputs)

# # Compile the model
# # optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
# optimizer = tf.keras.optimizers.Adadelta(learning_rate=0.1)
# model.compile(optimizer=optimizer, loss='mse')

# # Train the model
# model.fit(X_train, y_train, epochs=100, batch_size=32)


In [88]:
y_train.shape

(2500, 5)

In [89]:
X_train.shape

(2500, 19, 6523)

In [101]:
y_train

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]], dtype=float32)

In [90]:
import tensorflow as tf


# timesteps = X_train.shape[1]
# input_dim = X_train.shape[2]
# output_dim = 5

# # Define the model
# model = tf.keras.Sequential()

# # Add the recurrent layer (Elman network)
# model.add(tf.keras.layers.SimpleRNN(units=64, input_shape=(timesteps, input_dim)))

# # Add a dense (fully connected) layer for the output
# model.add(tf.keras.layers.Dense(units=output_dim, activation='softmax'))

# # Compile the model
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# # Print the model summary
# model.summary()

# # Train the model
# model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Define the Elman network architecture
input_dim = X_train.shape[2]
hidden_dim = 100
output_dim = y_train.shape[1]
num_steps = X_train.shape[1]

inputs = tf.keras.layers.Input(shape=(num_steps, input_dim))
hidden_layer = tf.keras.layers.SimpleRNN(hidden_dim, activation='relu')(inputs)
outputs = tf.keras.layers.Dense(output_dim, activation='relu')(hidden_layer)

# Define the model
model = tf.keras.Model(inputs=inputs, outputs=outputs)

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
# optimizer = tf.keras.optimizers.Adadelta(learning_rate=0.01)
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=31, validation_data=(X_test, y_test))


Epoch 1/20
81/81 [==============================] - 16s 155ms/step - loss: 0.1772 - accuracy: 0.2028 - val_loss: 0.1715 - val_accuracy: 0.2747
Epoch 2/20
81/81 [==============================] - 10s 124ms/step - loss: 0.1610 - accuracy: 0.3004 - val_loss: 0.1546 - val_accuracy: 0.3667
Epoch 3/20
81/81 [==============================] - 9s 111ms/step - loss: 0.1386 - accuracy: 0.4552 - val_loss: 0.1434 - val_accuracy: 0.4413
Epoch 4/20
81/81 [==============================] - 10s 123ms/step - loss: 0.1134 - accuracy: 0.6104 - val_loss: 0.1359 - val_accuracy: 0.4867
Epoch 5/20
81/81 [==============================] - 11s 139ms/step - loss: 0.0911 - accuracy: 0.7000 - val_loss: 0.1585 - val_accuracy: 0.4147
Epoch 6/20
81/81 [==============================] - 11s 138ms/step - loss: 0.0785 - accuracy: 0.7380 - val_loss: 0.1428 - val_accuracy: 0.5027
Epoch 7/20
81/81 [==============================] - 12s 144ms/step - loss: 0.0714 - accuracy: 0.7516 - val_loss: 0.1407 - val_accuracy: 0.5227


In [91]:
loss, acc = model.evaluate(X_test, y_test)

24/24 [==============================] - 2s 70ms/step - loss: 0.1226 - accuracy: 0.6547


In [92]:
import numpy as np

# Evaluate the model on the test data
loss = model.evaluate(X_test, y_test, verbose=0)

# Get the model's predictions on the test data
predictions = model.predict(X_test, verbose=0)

# Convert predictions and labels to class indices
# predicted_labels = np.argmax(predictions, axis=1)
predicted_labels = np.round(predictions)

# Compute accuracy
accuracy = np.mean(predicted_labels == y_test)

print("Accuracy:", np.round(accuracy * 100, 2))


Accuracy: 86.37


In [98]:
import pickle

with open('./models/65-86-model.p', 'wb') as outfile:
    pickle.dump(model, outfile)

In [94]:
predicted_labels

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.]], dtype=float32)

In [95]:
np.unique(predicted_labels, axis=0)

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.]], dtype=float32)

In [96]:
np.unique(y_test, axis=0)

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0.]], dtype=float32)